# **Brain Dead 2nd Problem Statement**
## ***BART Finetuned on Pubmed Dataset with QLoRA*** 

In [2]:
import warnings
warnings.filterwarnings("ignore")

### **Installing Required Libraries**

In [ ]:
!pip install kagglehub[hf-datasets]
!pip install datasets transformers peft sumy
!pip install --upgrade bitsandbytes
!pip install scikit-learn
!pip install rouge_score
!pip install datasets transformers peft sumy bitsandbytes evaluate rouge_score accelerate

### **Load CompScholar dataset (CSV with 371 rows)**

In [ ]:
import kagglehub
from kagglehub import KaggleDatasetAdapter
# Set the path to the file you'd like to load
file_path = "CompScholar.csv"
# Load the latest version
hf_dataset = kagglehub.load_dataset(
  KaggleDatasetAdapter.HUGGING_FACE,
  "akash01roy/compscholar",
  file_path,)
print("Hugging Face Dataset:", hf_dataset)

100%|██████████| 2.73M/2.73M [00:00<00:00, 3.36MB/s]

Hugging Face Dataset: Dataset({
    features: ['Paper Id', 'Paper Title', 'Key Words', 'Abstract', 'Conclusion', 'Document', 'Paper Type', 'Summary', 'Topic', 'OCR', 'labels'],
    num_rows: 371
})


In [24]:
compscholar_df = hf_dataset.to_pandas()
compscholar_df = compscholar_df[['Paper Title', 'Key Words', 'Paper Type', 'Topic', 'Document', 'Summary']]
compscholar_dataset = Dataset.from_pandas(compscholar_df)
print(f"CompScholar dataset: {compscholar_dataset.num_rows} samples")

CompScholar dataset: 371 samples


## **Section 1: Model Finetuning on PubMed Dataset**

In [ ]:
import os
import torch
import pandas as pd
from datasets import load_dataset, Dataset
from transformers import (
    BartTokenizer,
    BartForConditionalGeneration,
    TrainingArguments,
    Trainer,
    DataCollatorForSeq2Seq,
    GenerationConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer
)
from transformers import BitsAndBytesConfig
from peft import LoraConfig, get_peft_model, TaskType
import evaluate
from sklearn.model_selection import KFold

print("Loading PubMed dataset...")
pubmed_dataset = load_dataset('ccdv/pubmed-summarization')
train_df = pd.DataFrame(pubmed_dataset['train'])
train_df['article'] = train_df['article'].apply(lambda x: x.replace('\n', ''))
train_df = train_df.head(5000)
print(f"Working with {len(train_df)} samples for training.")


tokenizer = BartTokenizer.from_pretrained('facebook/bart-large')
max_input_length = 1024  
max_target_length = 256  

def preprocess_function(examples):
    model_inputs = tokenizer(
        examples['article'],
        max_length=max_input_length,
        truncation=True,
        padding='max_length'
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            examples['abstract'],
            max_length=max_target_length,
            truncation=True,
            padding='max_length'
        )
    model_inputs['labels'] = labels['input_ids']
    return model_inputs

train_dataset = Dataset.from_pandas(train_df)
train_dataset = train_dataset.map(preprocess_function, batched=True)

val_dataset = pubmed_dataset['validation'].select(range(1000))
val_dataset = val_dataset.map(preprocess_function, batched=True, remove_columns=['article', 'abstract'])

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4"
)
model = BartForConditionalGeneration.from_pretrained(
    "facebook/bart-large-cnn",
    quantization_config=bnb_config,
    device_map="auto",
    torch_dtype=torch.float16,
    low_cpu_mem_usage=True
)

lora_config = LoraConfig(
    task_type=TaskType.SEQ_2_SEQ_LM,
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj", "v_proj", "out_proj", "fc1", "fc2"]
)
model = get_peft_model(model, lora_config)
model.print_trainable_parameters()
print("Model loaded with QLoRA configuration successfully!")

data_collator = DataCollatorForSeq2Seq(
    tokenizer,
    model=model,
    label_pad_token_id=-100,
    pad_to_multiple_of=8
)

from transformers import TrainerCallback
class ProgressPercentageCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs):
        if state.max_steps and state.global_step:
            percent_complete = (state.global_step / state.max_steps) * 100
            print(f"Training progress: {percent_complete:.2f}% complete (step {state.global_step}/{state.max_steps})")
            if torch.cuda.is_available():
                for i in range(torch.cuda.device_count()):
                    gpu_mem = torch.cuda.memory_allocated(i) / (1024**3)
                    print(f"GPU {i} Memory: {gpu_mem:.2f} GB")
        return control

training_args = TrainingArguments(
    output_dir="./bart_pubmed_qlora_finetuned",  
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=1,
    learning_rate=5e-5,
    weight_decay=0.01,
    fp16=True,
    logging_steps=50,
    save_steps=200,        
    save_total_limit=2,
    evaluation_strategy="steps",
    eval_steps=200,
    no_cuda=False,
    report_to="none"
)

print("Training arguments configured successfully!")

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
    tokenizer=tokenizer,
)
trainer.add_callback(ProgressPercentageCallback)

print("Starting training on the PubMed dataset...")
trainer.train()
print("Training on the PubMed dataset completed!")

Loading PubMed dataset...
Working with 5000 samples for training.


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


trainable params: 4,325,376 || all params: 410,615,808 || trainable%: 1.0534
Model loaded with QLoRA configuration successfully!
Training arguments configured successfully!
Starting training on the PubMed dataset...


Step,Training Loss,Validation Loss
200,2.423600,2.187954


Training progress: 16.03% complete (step 50/312)
GPU 0 Memory: 0.34 GB
Training progress: 32.05% complete (step 100/312)
GPU 0 Memory: 0.34 GB
Training progress: 48.08% complete (step 150/312)
GPU 0 Memory: 0.34 GB
Training progress: 64.10% complete (step 200/312)
GPU 0 Memory: 0.34 GB
Training progress: 64.10% complete (step 200/312)
GPU 0 Memory: 0.34 GB
Training progress: 80.13% complete (step 250/312)
GPU 0 Memory: 0.34 GB
Training progress: 96.15% complete (step 300/312)
GPU 0 Memory: 0.34 GB
Training progress: 100.00% complete (step 312/312)
GPU 0 Memory: 0.34 GB
Training on the PubMed dataset completed!


## **Section 2: Inference on the CompScholar Dataset**

In [ ]:
base_prompt = """<s>[INST]
<<SYS>>
{system_prompt}
<</SYS>>

{user_prompt}[/INST]"""

system_prompt = ("Analyze the research article content and get me a summary from the research article. "
                 "The summary length may be within 150 words")

generation_config = GenerationConfig(
    max_new_tokens=150,
    num_beams=4,
    early_stopping=True,
    no_repeat_ngram_size=3,
)
generation_config.decoder_start_token_id = tokenizer.bos_token_id

def generate_summary(document):
    formatted_input = base_prompt.format(system_prompt=system_prompt, user_prompt=document)
    inputs = tokenizer(
        formatted_input,
        max_length=1024,
        truncation=True,
        return_tensors="pt"
    ).to(model.device)
    outputs = model.generate(input_ids=inputs.input_ids, **generation_config.to_dict())
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

comp_df = compscholar_df  
rouge_metric = evaluate.load("rouge")
bleu_metric = evaluate.load("bleu")

total_rouge1 = total_rouge2 = total_rougeL = total_bleu = 0
num_examples = len(comp_df)

for idx, row in comp_df.iterrows():
    document = row["Document"]
    truth_summary = row["Summary"]
    
    model_summary = generate_summary(document)
    
    print(f"\n=== Example {idx+1} ===")
    print(f"\nDOCUMENT:\n{document[:500]}...")
    print(f"\nGROUND TRUTH SUMMARY:\n{truth_summary}")
    print(f"\nMODEL GENERATED SUMMARY:\n{model_summary}")
    
    rouge_score = rouge_metric.compute(predictions=[model_summary], references=[truth_summary])
    bleu_score = bleu_metric.compute(predictions=[model_summary], references=[[truth_summary]])
    
    total_rouge1 += rouge_score["rouge1"]
    total_rouge2 += rouge_score["rouge2"]
    total_rougeL += rouge_score["rougeL"]
    total_bleu += bleu_score["bleu"]

avg_rouge1 = total_rouge1 / num_examples
avg_rouge2 = total_rouge2 / num_examples
avg_rougeL = total_rougeL / num_examples
avg_bleu = total_bleu / num_examples

print("\n" + "="*50 + "\nFinal Average Metrics on CompScholar Dataset\n" + "="*50)
print(f"Average ROUGE-1: {avg_rouge1:.4f}")
print(f"Average ROUGE-2: {avg_rouge2:.4f}")
print(f"Average ROUGE-L: {avg_rougeL:.4f}")
print(f"Average BLEU: {avg_bleu:.4f}")

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)
Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 1 ===

DOCUMENT:
Multi-document Summarization via Deep Learning
Techniques: A SurveyMulti-document summarization (MDS), Deep learning models, Objective functions, Taxonomy, Evaluation metrics, Future directions, Information aggregationMulti-document summarization (MDS) is an effective tool for information aggregation that generates an informative and concise summary from a cluster of topic-related documents. Our survey, the first of its kind,
systematically overviews the recent deep-learning-based MDS models. ...

GROUND TRUTH SUMMARY:
This article presents a systematic overview of recent deep-learning-based models for multi-document summarization (MDS). It proposes a new taxonomy to categorize network design strategies and provides an overview of objective functions, evaluation metrics, and datasets. The article also discusses open problems and future research directions in the field. The survey aims to provide a comprehensive understanding of MDS tasks and highlight nota

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 2 ===

DOCUMENT:
NLP based Machine Learning Approaches for Text 
SummarizationText summarization, Abstractive and extractive summaries, Query-based summarization, Structured-based and semantic-based approaches, Evaluation metrics (ROGUE score and TF_IDF score), Future directions (GANs and transfer learning),Data abundance and information overloadDue to the plethora of data available today, text 
summarization has become very essential to gain just the right 
amount of information from huge texts. We see long ...

GROUND TRUTH SUMMARY:
The article discusses the importance of text summarization due to the abundance of data available today. Various approaches, including abstractive and extractive methods, as well as query-based techniques, are presented. The paper focuses on structured and semantic-based approaches and discusses the datasets used for testing these methods. While the accuracy of the summaries generated using these methods can be compared, there is no specific 

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)


...


=== Example 368 ===

DOCUMENT:
Object semantics sentiment correlation analysis enhanced image sentiment classificationImage sentiment classification, Object semantics, Bayesian network, Object semantics sentiment correlation model, Convolutional Neural NetworkWith the development of artificial intelligence and deep learning, image sentiment analysis has become a hotspot in computer vision and attracts more attention. Most of the existing methods focus on identifying the emotions by studying complex models or robust features f...

GROUND TRUTH SUMMARY:
The text proposes a new method for image sentiment analysis that uses the correlation between object semantics and image sentiment to improve accuracy. The proposed method involves a Bayesian network model that represents the correlation between image emotions and object semantics, resulting in a probability distribution for object semantics combinations. Experiments on popular datasets show that the proposed method outperforms existing 

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 369 ===

DOCUMENT:
Identification of fact-implied implicit sentiment based on multi-level semantic fused representationFact-implied implicit sentiment, Multi-level feature fusion, Representation learning, Sentiment analysis, Tree convolutionSentiment can be expressed in an explicit or implicit manner. Most of the current studies on sentiment analysis focus on the identification of explicit sentiment but ignore the implicit. According to our statistics during data labeling in previous work, nearly a third of subjec...

GROUND TRUTH SUMMARY:
The paper presents a method for identifying and classifying the polarity of implicit sentiment sentences that are conveyed through facts. The authors define implicit sentiment and propose a multi-level semantic feature fusion model that considers word-level sentiment, sentence-level implicit sentiment, and document-level context. The proposed method achieved high accuracy in identifying and classifying implicit sentiment polarity in two 

Both `max_new_tokens` (=150) and `max_length`(=20) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



=== Example 370 ===

DOCUMENT:
Salient object based visual sentiment analysis by combining deep features and handcrafted featuresSalient object detection, sentiment analysis, feature fusion, Convolutional Neural Network With the rapid growth of social networks, the visual sentiment analysis has quickly emerged for opinion mining. Recent study reveals that the sentiments conveyed by some images are related to salient objects in them, we propose a scheme for visual sentiment analysis that combines deep and handcrafted features....

GROUND TRUTH SUMMARY:
Based on the proposed framework, the sentiment classification of a given image can be summarized as follows. For a given image, salient objects are first generated. In order to reduce redundancy of salient objects for a single image, the candidate selection method is applied based on their sentiment scores and the best candidates are kept. Deep features are extracted using pretrained model and handcrafted features such as visual texture,

## **Section 3: Further Fine-Tuning on the CompScholar Dataset using K-Fold Cross-Validation**

In [ ]:
import copy
comp_df = compscholar_df[['Document', 'Summary']]
comp_dataset = Dataset.from_pandas(comp_df)

kf = KFold(n_splits=3, shuffle=True, random_state=42)
cv_results = []

for fold, (train_index, val_index) in enumerate(kf.split(comp_dataset)):
    print(f"\n========== Starting Fold {fold+1}/3 ==========")
    train_fold = comp_dataset.select(train_index)
    val_fold = comp_dataset.select(val_index)
    import copy
    model_fold = copy.deepcopy(model) 
    model_fold.print_trainable_parameters()
    
    max_input_length = 1024
    max_target_length = 128

    def preprocess_comp(examples):
        inputs = examples["Document"]
        targets = examples["Summary"]
        model_inputs = tokenizer(inputs, max_length=max_input_length, truncation=True)
        labels = tokenizer(text_target=targets, max_length=max_target_length, truncation=True)
        model_inputs["labels"] = labels["input_ids"]
        return model_inputs

    train_fold = train_fold.map(preprocess_comp, batched=True, remove_columns=train_fold.column_names)
    val_fold = val_fold.map(preprocess_comp, batched=True, remove_columns=val_fold.column_names)
    
    data_collator_fold = DataCollatorForSeq2Seq(tokenizer, model=model_fold)
    rouge_metric_fold = evaluate.load("rouge")
    
    def compute_metrics(eval_pred):
        predictions, labels = eval_pred
        decoded_preds = tokenizer.batch_decode(predictions, skip_special_tokens=True)
        labels = [[(l if l != -100 else tokenizer.pad_token_id) for l in label] for label in labels]
        decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)
        return rouge_metric_fold.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    
    training_args_fold = Seq2SeqTrainingArguments(
        output_dir=f"./fold_{fold+1}_bart_finetuned",  
        evaluation_strategy="steps",
        eval_steps=500,
        learning_rate=5e-5,
        per_device_train_batch_size=2,
        per_device_eval_batch_size=2,
        weight_decay=0.01,
        save_total_limit=1,
        num_train_epochs=3,
        predict_with_generate=True,
        fp16=True,
        logging_steps=100,
        save_steps=500,
    )
    
    trainer_fold = Seq2SeqTrainer(
        model=model_fold,
        args=training_args_fold,
        train_dataset=train_fold,
        eval_dataset=val_fold,
        tokenizer=tokenizer,
        data_collator=data_collator_fold,
        compute_metrics=compute_metrics,
    )
    
    trainer_fold.train()
    eval_result = trainer_fold.evaluate()
    print(f"Fold {fold+1} evaluation result:")
    print(eval_result)
    cv_results.append(eval_result)

best_fold = None
best_rouge1 = 0
for i, res in enumerate(cv_results):
    rouge1 = res.get("eval_rouge1", 0)
    if rouge1 > best_rouge1:
        best_rouge1 = rouge1
        best_fold = i + 1

print(f"\nBest fold is Fold {best_fold} with ROUGE-1: {best_rouge1:.4f}")
for i, res in enumerate(cv_results):
    print(f"Fold {i+1}: ROUGE-1: {res.get('eval_rouge1', 0):.4f}, ROUGE-2: {res.get('eval_rouge2', 0):.4f}, ROUGE-L: {res.get('eval_rougeL', 0):.4f}")


========== Starting Fold 1/3 ==========
trainable params: 4,325,376 || all params: 410,615,808 || trainable%: 1.0534


Map:   0%|          | 0/247 [00:00<?, ? examples/s]

Map:   0%|          | 0/124 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


...

Map:   0%|          | 0/248 [00:00<?, ? examples/s]

Map:   0%|          | 0/123 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSeq2SeqLM`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Step,Training Loss,Validation Loss


Fold 3 evaluation result:
{'eval_loss': 1.0183215141296387, 'eval_rouge1': 0.5953908501468936, 'eval_rouge2': 0.3375410242187876, 'eval_rougeL': 0.41570193390866184, 'eval_rougeLsum': 0.4158769049092712, 'eval_runtime': 299.5259, 'eval_samples_per_second': 0.411, 'eval_steps_per_second': 0.207, 'epoch': 3.0}

Best fold is Fold 2 with ROUGE-1: 0.6138
Fold 1: ROUGE-1: 0.6108, ROUGE-2: 0.3458, ROUGE-L: 0.4297
Fold 2: ROUGE-1: 0.6138, ROUGE-2: 0.3654, ROUGE-L: 0.4457
Fold 3: ROUGE-1: 0.5954, ROUGE-2: 0.3375, ROUGE-L: 0.4157
